In [1]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df["label"].unique()

In [ ]:
train_df

In [4]:
train_df = train_df[train_df["label"].isin(["linje1", "linje2", "linje3", "linje4", "linje5", "linje6"])]
test_df = test_df[test_df["label"].isin(["linje1", "linje2", "linje3", "linje4", "linje5", "linje6"])]

In [ ]:
train_df

In [ ]:
train_df[train_df["task_id"] == 4287]

In [7]:
train_drawings = train_df.groupby('task_id').first().reset_index()
test_drawings = test_df.groupby('task_id').first().reset_index()

In [8]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np

def visualize_image_with_bounding_boxes_colored(df):
    """
    Visualizes an image with bounding boxes, each colored according to the rectanglelabels.
    Adds labels and colors to the legend.
    
    Parameters:
    - image_path: Path to the image file.
    - bounding_boxes: A list of bounding boxes, where each bounding box is represented as a dictionary
      with keys 'x', 'y', 'width', 'height', and optionally 'label'. Coordinates are normalized to [0, 1].
    """
    # Open the image file
    img = Image.open(df["image"])
    fig, ax = plt.subplots(1)
    fig.set_size_inches(10, 10)
    ax.imshow(img)
    
    # Image dimensions
    img_width, img_height = img.size
    
    # Label to color mapping
    label_color_map = {
        'riktning_text': 'r',
        'another_label': 'g',
          # Example additional label
        # Add more labels and colors as needed
    }
    

    # Denormalize coordinates
    x = df['x'] * img_width //100
    y = df['y'] * img_height //100
    width = df['width'] * img_width //100
    height = df['height'] * img_height //100
    
    # Get color for the label
    label = df['label']
    color = label_color_map.get(label, 'b')  # Default to blue if label not in map
    
    # Create a Rectangle patch
    rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor=color, facecolor='none', label=label)
    
    # Add the patch to the Axes
    ax.add_patch(rect)
    
    # Create legend from unique labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))  # Removing duplicates
    ax.legend(by_label.values(), by_label.keys())
    
    return plt

In [ ]:
visualize_image_with_bounding_boxes_colored(train_df.iloc[0]).show()

In [10]:
#data loader for the dataset
import torch
from PIL import Image
from torchvision.transforms import functional as F
import os
import cv2

class KBABygglovDataset(torch.utils.data.Dataset):
    def __init__(self, drawings, df,scale = 1):
        self.df = df
        self.drawings = drawings
        self.transforms = None
        self.scale = scale
        #self.task_ids = df["task_id"].unique()
    
    def __len__(self):
        return len(self.drawings)
    
    def __getitem__(self, idx):
        task_id = self.drawings.iloc[idx]["task_id"]
        rows = self.df[self.df["task_id"] == task_id]
        image = cv2.imread(rows.iloc[0]["image"])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB ) 
        #contour = find_bounding_box(image)
        
        
        # Normalize bounding box coordinates
        width, height = rows.iloc[0]["original_width"], rows.iloc[0]["original_height"]
        target = {}
        boxes = []
        labels = []
        lines = ["linje1", "linje2", "linje3", "linje4", "linje5", "linje6"]
        for line in lines:
            line_rows = rows[rows["label"] == line]
            if len(line_rows) == 0:
                continue
            x1 = line_rows["x"].min()*width//100 - 20
            y1 = line_rows["y"].min()*height//100 - 20
            x2 = line_rows["x"].max()*width//100 + 20
            y2 = line_rows["y"].max()*height//100 + 20
            boxes.append([x1, y1, x2, y2])
            labels.append(1)
        target["boxes"] = torch.as_tensor(boxes, 
                                dtype = torch.float32)
        target["labels"]=torch.as_tensor(labels,
                        dtype = torch.int64)
        target["image_id"] = torch.as_tensor([task_id])
        #scale the image and target to half size
        image = cv2.resize(image, (image.shape[1]// self.scale,  image.shape[0]// self.scale))

        target["boxes"] = target["boxes"]/ self.scale

        image = F.to_tensor(image)
        #cropped_image, target = crop_to_contour(image, target, contour)
        return image, target




dataset = KBABygglovDataset(train_drawings, train_df)
test_dataset = KBABygglovDataset(test_drawings, test_df)

In [ ]:
len(dataset)

In [ ]:
len(test_dataset)

In [ ]:
for data in dataset:
    image, boxes = data
    image.to("cpu")
    print(boxes)
    print(image.shape)
    fig, ax = plt.subplots(1)
    fig.set_size_inches(10, 10)
    ax.imshow(image.permute(1, 2, 0))
    for box in boxes["boxes"]:
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()
    break

In [14]:
# Collate image-target pairs into a tuple.
def collate_fn(batch):
    return tuple(zip(*batch))
# Create the DataLoaders from the Datasets. 
train_dl = torch.utils.data.DataLoader(dataset, 
                                 batch_size = 4, 
                                 shuffle = True, 
                        collate_fn = collate_fn)
'''val_dl = torch.utils.data.DataLoader(val_ds, 
                             batch_size = 4, 
                            shuffle = False, 
                    collate_fn = collate_fn)'''
test_dl = torch.utils.data.DataLoader(test_dataset, 
                               batch_size = 1, 
                              shuffle = False, 
                      collate_fn = collate_fn)

In [15]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
def get_object_detection_model(num_classes = 2):

    model = fasterrcnn_resnet50_fpn(pretrained = False)

    # Replace the original 91 class top layer with a new layer
    # tailored for num_classes.
    in_feats = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_feats,
                                                   num_classes)
    return model

In [16]:
def unbatch(batch, device):
    """
    Unbatches a batch of data from the Dataloader.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        X: list
            List of images.
        y: list
            List of dictionaries.
    """
    X, y = batch
    X = [x.to(device) for x in X]
    y = [{k: v.to(device) for k, v in t.items()} for t in y]
    return X, y
def train_batch(batch, model, optimizer, device):
    """
    Uses back propagation to train a model.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()
    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    return loss, losses
@torch.no_grad()
def validate_batch(batch, model, optimizer, device):
    """
    Evaluates a model's loss value using validation data.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()

    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    return loss, losses

In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
from torchvision.transforms import functional as F
import torch_snippets.torch_loader as tl


num_epochs =10
# Assuming 'dataset' is an instance of 'KBABygglovDataset' and 'data_loader' is an instance of 'DataLoader'
# Also assuming 'device' is defined (e.g., cuda or cpu)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_object_detection_model(num_classes = 2)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, 
                        lr = 0.005, 
                    momentum = 0.9, 
             weight_decay = 0.0005)


log = tl.Report(num_epochs)
# FasterRCNN loss names.
keys = ["loss_classifier", 
            "loss_box_reg", 
        "loss_objectness", 
        "loss_rpn_box_reg"]
model.to(device)
for epoch in range(num_epochs):
    N = len(train_dl)
    for ix, batch in enumerate(train_dl):
        loss, losses = train_batch(batch, model, 
                                optimizer, device)
        # Record the current train loss.
        pos = epoch + (ix + 1) / N
        log.record(pos = pos, trn_loss = loss.item(), 
                    end = "\r")
    if test_dl is not None:
        N = len(test_dl)
        for ix, batch in enumerate(test_dl):
            loss, losses = validate_batch(batch, model, 
                                        optimizer, device)
            
            # Record the current validation loss.
            pos = epoch + (ix + 1) / N
            log.record(pos = pos, val_loss = loss.item(), 
                        end = "\r")
log.report_avgs(epoch + 1)
log


In [18]:
torch.save(model, 'distance_marker.model')

In [19]:
@torch.no_grad()
def predict_batch(batch, model, device):
    model.to(device)
    model.eval()
    X, _ = unbatch(batch, device = device)
    predictions = model(X)
    return [x.cpu() for x in X], predictions
def predict(model, data_loader, device = "cpu"):
    images = []
    predictions = []
    for i, batch in enumerate(data_loader):
        X, p = predict_batch(batch, model, device)
        images.append(X)
        predictions.append(p)
    
    return images, predictions

In [20]:

images, predictions = predict(model, test_dl, device = device)

In [21]:
import torchvision
def decode_prediction(prediction, 
                      score_threshold = 0.8, 
                      nms_iou_threshold = 0.2):
    """
    Inputs
        prediction: dict
        score_threshold: float
        nms_iou_threshold: float
    Returns
        prediction: tuple
    """
    boxes = prediction["boxes"]
    scores = prediction["scores"]
    labels = prediction["labels"]
    # Remove any low-score predictions.
    if score_threshold is not None:
        want = scores > score_threshold
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]
    # Remove any overlapping bounding boxes using NMS.
    if nms_iou_threshold is not None:
        want = torchvision.ops.nms(boxes = boxes, scores = scores, 
                                iou_threshold = nms_iou_threshold)
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]
    return (boxes.cpu().numpy(), 
            labels.cpu().numpy(), 
            scores.cpu().numpy())

In [22]:
def calculate_iou(box1, box2):
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Calculate the coordinates of the intersection rectangle
    x_left = max(x1_1, x1_2)
    y_top = max(y1_1, y1_2)
    x_right = min(x2_1, x2_2)
    y_bottom = min(y2_1, y2_2)
    
    # Calculate the area of the intersection rectangle
    if x_right < x_left or y_bottom < y_top:
        return 0.0  # No overlap
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    # Calculate the areas of both bounding boxes
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Calculate the union area
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate the IoU
    iou = intersection_area / union_area
    return iou

def eliminate_overlapping_boxes(boxes, overlap_threshold=0.8):
    # Sort the boxes by area (optional, can improve performance)
    boxes = sorted(boxes, key=lambda b: (b[2] - b[0]) * (b[3] - b[1]), reverse=True)
    
    keep_boxes = []
    while boxes:
        # Take the first box and compare it with the rest
        current_box = boxes.pop(0)
        keep = True
        
        for other_box in keep_boxes:
            iou = calculate_iou(current_box, other_box)
            if iou > overlap_threshold:
                keep = False
                break
        
        if keep:
            keep_boxes.append(current_box)
    
    return keep_boxes


In [25]:
def visualize_image_with_bounding_boxes_colored_img(img, bounding_boxes):
    """
    Visualizes an image with bounding boxes, each colored according to the rectanglelabels.
    Adds labels and colors to the legend.
    
    Parameters:
    - image_path: Path to the image file.
    - bounding_boxes: A list of bounding boxes, where each bounding box is represented as a dictionary
      with keys 'x', 'y', 'width', 'height', and optionally 'label'. Coordinates are normalized to [0, 1].
    """
    # Open the image file
    fig, ax = plt.subplots(1)
    fig.set_size_inches(10, 10)
    ax.imshow(img.permute(1, 2, 0).numpy())
    
    # Image dimensions
    img_width, img_height = img.shape[0], img.shape[1]
    
    # Label to color mapping
    label_color_map = {
        'riktning_text': 'r',
        'another_label': 'g',  # Example additional label
        # Add more labels and colors as needed
    }
    print(bounding_boxes["scores"].cpu())
    if len(bounding_boxes["scores"].cpu()) != 0:
        print(np.argmax(bounding_boxes["scores"].cpu()))
        # Add bounding boxes
        for i in range(len(bounding_boxes["boxes"].cpu())):
            # Denormalize coordinates
            if bounding_boxes["scores"].cpu()[i] < 0.5:
                continue
            box = bounding_boxes["boxes"].cpu()[i]

            print(box)
            x = box[0] 
            y = box[1]
            width = box[2]-box[0]
            height = box[3]-box[1]
            
            # Create a Rectangle patch
            rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor="red", facecolor='none', label=1)
            
            # Add the patch to the Axes
            ax.add_patch(rect)
    
    # Create legend from unique labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))  # Removing duplicates
    ax.legend(by_label.values(), by_label.keys())
    
    plt.show()

In [ ]:
#from Object_detection.object_detection_helper import visualize_image_with_bounding_boxes_colored_img
for i in range(len(images)):
    print(i)
    
    visualize_image_with_bounding_boxes_colored_img(images[i][0], predictions[i][0])
